# 1. Import Ntuple and DecayHash

In [1]:
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import sys
sys.path.append('/home/belle/zhangboy/inclusive_R_D/')
import utilities as util
import decayHash
from decayHash import DecayHashMap

Welcome to JupyROOT 6.24/06


In [8]:
# read a signal mc as a pandas dataframe
files = ['sigDDst', 'normDDst','bkgDststp_tau', 'bkgDstst0_tau','bkgDstst0_ell']

file_name=files[2]
filename=f'/home/belle/zhangboy/inclusive_R_D/Samples/Signal_MC14ri/MC14ri_{file_name}_bengal_e_2/{file_name}_bengal_e_2_0.parquet'
data = pd.read_parquet(filename, engine="pyarrow")
df, samples=util.get_dataframe_samples(data)
decayhash=f'/home/belle/zhangboy/inclusive_R_D/Samples/Signal_MC14ri/MC14ri_{file_name}_bengal_e_2/hashmap_{file_name}_bengal_e_2.root'
hashmap2 = DecayHashMap(decayhash, removeRadiativeGammaFlag=True)

for name in samples:
    print(f'{name}: {len(samples[name])}')

$D\tau\nu$: 1555
$D^\ast\tau\nu$: 978
$D\ell\nu$: 39547
$D^\ast\ell\nu$: 29703
$D^{\ast\ast}\tau\nu$: 52628
$D^{\ast\ast}\ell\nu$: 12558
bkg_fakeDTC: 510320
bkg_fakeB: 82981
bkg_continuum: 0
bkg_others: 1365


In [ ]:
import pandas
pandas.set_option('display.max_rows', None)
print(data.isna().sum())

In [14]:
candidate12 = samples[r'$D\ell\nu$'].iloc[1111][['B0_DecayHash', "B0_DecayHashEx"]].values

# print the original decay as simulated in MC with removed Bremsstrahlung gammas
print("Monte Carlo Decay with removed Bremsstrahlung gammas: ")
org2 = hashmap2.get_original_decay(*candidate12)
print(org2.to_string())

Monte Carlo Decay with removed Bremsstrahlung gammas: 
300553
    511
        -415
            -421
                310
                211
                -211
            -211
        -15
            323
                311
                    310
                211
            -16
        16
    -511
        411
            -321
            211
            211
        11
        -12



In [49]:
a=df_bestSelected.query('e_mcErrors==2048')[['e_mcPDG','e_genMotherPDG','D_mcErrors','D_mcPDG','B0_mcErrors','B0_mcPDG']].value_counts()
a[a>120]

e_mcPDG  e_genMotherPDG  D_mcErrors  D_mcPDG    B0_mcErrors  B0_mcPDG 
-11.0     511.0          0.0         -411.0     24.0          511.0       4357
                         312.0        300553.0  312.0         300553.0    4313
 11.0    -511.0          0.0          411.0     24.0         -511.0       4199
-11.0     511.0          0.0         -411.0     8.0           511.0       4180
 11.0    -511.0          312.0        300553.0  312.0         300553.0    4164
                         0.0          411.0     8.0          -511.0       4022
-11.0     511.0          376.0        300553.0  376.0         300553.0    2434
 11.0    -511.0          376.0        300553.0  376.0         300553.0    2355
-11.0     511.0          440.0        300553.0  440.0         300553.0    1894
 11.0    -511.0          440.0        300553.0  440.0         300553.0    1783
-11.0     511.0          504.0        300553.0  504.0         300553.0    1263
 11.0    -511.0          504.0        300553.0  504.0       